# Goal of the notebook

This notebook is used to create more advanced features. For now we are sure that additional features will improve the performance of models (especially time series features, such as lags). Besides them, we will add features related to companies (such as their age or weight in index). As a result, we will end up with a new version of the dataset.

In [30]:
import pandas as pd
import numpy as np

In [31]:
dataset_full = pd.read_csv('5_Fair_volatility_data.csv')
dataset_full

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (3,4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Contract Name,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Company,Date_get_data,Date_expiration,Time_to_maturity,r,Stock_price,Stock_volume,Black_Scholes,Option_price_volatility
0,AAPL240301C00095000,95.0,93.48,87.2,87.95,0.0,-,-,1.0,AAPL,2024-02-16,2024-03-01,0.038356,0.04,182.309998,49701400.0,96,3.64518
1,AAPL240301C00095000,95.0,93.48,87.2,87.95,0.0,-,-,1.0,AAPL,2024-02-19,2024-03-01,0.030137,0.04,182.309998,49701400.0,93,3.64518
2,AAPL240301C00095000,95.0,93.48,85.8,86.2,0.0,-,-,1.0,AAPL,2024-02-20,2024-03-01,0.027397,0.04,181.559998,53665600.0,92,3.64518
3,AAPL240301C00095000,95.0,93.48,86.4,86.7,0.0,-,-,1.0,AAPL,2024-02-21,2024-03-01,0.024658,0.04,182.320007,41529700.0,92,3.64518
4,AAPL240301C00095000,95.0,93.48,88.75,89.1,0.0,-,-,1.0,AAPL,2024-02-22,2024-03-01,0.021918,0.04,184.369995,52292200.0,93,3.64518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167986,ORCL250620C00065000,65.0,63.17,61.0,63.6,0.0,-,11,11,ORCL,2024-04-08,2025-06-20,1.200000,0.04,124.349998,6119200.0,62,0.00000
1167987,ORCL250620C00065000,65.0,63.17,59.45,61.1,0.0,-,11,11,ORCL,2024-04-09,2025-06-20,1.197260,0.04,123.230003,5564900.0,61,0.00000
1167988,ORCL250620C00065000,65.0,63.17,59.0,61.75,0.0,-,11,11,ORCL,2024-04-10,2025-06-20,1.194521,0.04,121.750000,4509900.0,59,0.00000
1167989,ORCL250620C00065000,65.0,63.17,60.3,62.6,0.0,-,11,11,ORCL,2024-04-11,2025-06-20,1.191781,0.04,123.239998,7315200.0,61,0.00000


In [32]:
dataset_full['Contract Name'].nunique()

37223

In [33]:
dataset_full.groupby('Contract Name')

### Get features related to companies

https://www.slickcharts.com/sp500 - weights

https://companiesmarketcap.com - capitalizations

Ages - information might not be very precise (but still the best we can do)

Another limitation of study - some useful features (news, dynamics of stocks in nearest past) are not used

In [34]:
# capitalizations in $billions
market_capitalizations = [['AAPL', 2831.0], 
            ['GOOGL', 2076.0],
            ['MSFT', 3022.0],
            ['SPY', 0],
            ['PFE', 157.47],
            ['LMT', 110.82],
            ['DIS', 208.48],
            ['VZ', 163.69],
            ['PYPL', 68.72],
            ['TSN', 22.14],
            ['DG', 30.20],
            ['QCOM', 200.47],
            ['META', 1146.0],
            ['TSLA', 577.85],
            ['NFLX', 249.63],
            ['NVDA', 2219.0],
            ['AMZN', 1937.0],
            ['AVGO', 592.30],
            ['LLY', 698.51],
            ['JPM', 547.08],
            ['UNH', 453.09],
            ['V', 549.35],
            ['MA', 412.35],
            ['JNJ', 359.24],
            ['PG', 388.14],
            ['MRK', 322.98],
            ['MCD', 194.90],
            ['WMT', 482.13],
            ['BAC', 291.30],
            ['ACN', 190.95],
            ['KO', 267.82],
            ['PEP', 242.16],
            ['CSCO', 190.79],
            ['ORCL', 318.27],
            ['INTC', 133.71]]

In [35]:
capitalizations = pd.DataFrame(market_capitalizations, columns=['Company', 'Capitalization'])
capitalizations.head()

,Company,Capitalization
0,AAPL,2831.00
1,GOOGL,2076.00
2,MSFT,3022.00
3,SPY,0.00
4,PFE,157.47


In [36]:
# substitute 0 for S&P 500 with the median
med_capitalization = capitalizations['Capitalization'].median()
capitalizations.loc[capitalizations['Company'] == 'SPY', 'Capitalization'] = med_capitalization
capitalizations.head()

,Company,Capitalization
0,AAPL,2831.00
1,GOOGL,2076.00
2,MSFT,3022.00
3,SPY,318.27
4,PFE,157.47


In [37]:
ages_company = [['AAPL', 48], 
            ['GOOGL', 26],
            ['MSFT', 49],
            ['SPY', 0],
            ['PFE', 175],
            ['LMT', 29],
            ['DIS', 101],
            ['VZ', 24],
            ['PYPL', 26],
            ['TSN', 31],
            ['DG', 85],
            ['QCOM', 39],
            ['META', 20],
            ['TSLA', 21],
            ['NFLX', 27],
            ['NVDA', 31],
            ['AMZN', 29],
            ['AVGO', 63],
            ['LLY', 148],
            ['JPM', 24],
            ['UNH', 57],
            ['V', 65],
            ['MA', 64],
            ['JNJ', 138],
            ['PG', 193],
            ['MRK', 133],
            ['MCD', 84],
            ['WMT', 62],
            ['BAC', 26],
            ['ACN', 35],
            ['KO', 137],
            ['PEP', 131],
            ['CSCO', 40],
            ['ORCL', 47],
            ['INTC', 56]]

In [38]:
ages = pd.DataFrame(ages_company, columns=['Company', 'Age'])
ages.head()

,Company,Age
0,AAPL,48
1,GOOGL,26
2,MSFT,49
3,SPY,0
4,PFE,175


In [39]:
# substitute 0 for S&P 500 with the median 
med_ages = ages['Age'].median()
ages.loc[ages['Company'] == 'SPY', 'Age'] = med_ages
ages.head()

,Company,Age
0,AAPL,48
1,GOOGL,26
2,MSFT,49
3,SPY,48
4,PFE,175


In [40]:
# weights in S&P 500 in %
weights_in_index = [['AAPL', 5.91], 
            ['GOOGL', 2.31],
            ['MSFT', 6.95],
            ['SPY', 0],
            ['PFE', 0.37],
            ['LMT', 0.23],
            ['DIS', 0.48],
            ['VZ', 0.38],
            ['PYPL', 0.17],
            ['TSN', 0.04],
            ['DG', 0.07],
            ['QCOM', 0.47],
            ['META', 2.29],
            ['TSLA', 1.17],
            ['NFLX', 0.58],
            ['NVDA', 4.98],
            ['AMZN', 3.97],
            ['AVGO', 1.28],
            ['LLY', 1.42],
            ['JPM', 1.30],
            ['UNH', 1.07],
            ['V', 1.00],
            ['MA', 0.86],
            ['JNJ', 0.85],
            ['PG', 0.91],
            ['MRK', 0.76],
            ['MCD', 0.47],
            ['WMT', 0.60],
            ['BAC', 0.60],
            ['ACN', 0.44],
            ['KO', 0.57],
            ['PEP', 0.57],
            ['CSCO', 0.45],
            ['ORCL', 0.43],
            ['INTC', 0.30]]

In [41]:
weights = pd.DataFrame(weights_in_index, columns=['Company', 'Weight'])
weights.head()

,Company,Weight
0,AAPL,5.91
1,GOOGL,2.31
2,MSFT,6.95
3,SPY,0.00
4,PFE,0.37


In [42]:
# substitute 0 for S&P 500 with the median
med_weight = weights['Weight'].median()
weights.loc[weights['Company'] == 'SPY', 'Weight'] = med_weight
weights.head()

,Company,Weight
0,AAPL,5.91
1,GOOGL,2.31
2,MSFT,6.95
3,SPY,0.60
4,PFE,0.37


In [43]:
add_features = capitalizations.merge(ages, on='Company')
add_features = add_features.merge(weights, on='Company')

# check that there were no mistakes while merging
assert len(add_features) == len(weights) == len(capitalizations) == len(ages), 'MISMATCH'

In [44]:
add_features.head()

,Company,Capitalization,Age,Weight
0,AAPL,2831.00,48,5.91
1,GOOGL,2076.00,26,2.31
2,MSFT,3022.00,49,6.95
3,SPY,318.27,48,0.60
4,PFE,157.47,175,0.37


In [45]:
dataset_full.head()

,Contract Name,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Company,Date_get_data,Date_expiration,Time_to_maturity,r,Stock_price,Stock_volume,Black_Scholes,Option_price_volatility
0,AAPL240301C00095000,95.0,93.48,87.2,87.95,0.0,-,-,1.0,AAPL,2024-02-16,2024-03-01,0.038356,0.04,182.309998,49701400.0,96,3.64518
1,AAPL240301C00095000,95.0,93.48,87.2,87.95,0.0,-,-,1.0,AAPL,2024-02-19,2024-03-01,0.030137,0.04,182.309998,49701400.0,93,3.64518
2,AAPL240301C00095000,95.0,93.48,85.8,86.2,0.0,-,-,1.0,AAPL,2024-02-20,2024-03-01,0.027397,0.04,181.559998,53665600.0,92,3.64518
3,AAPL240301C00095000,95.0,93.48,86.4,86.7,0.0,-,-,1.0,AAPL,2024-02-21,2024-03-01,0.024658,0.04,182.320007,41529700.0,92,3.64518
4,AAPL240301C00095000,95.0,93.48,88.75,89.1,0.0,-,-,1.0,AAPL,2024-02-22,2024-03-01,0.021918,0.04,184.369995,52292200.0,93,3.64518


In [46]:
length = len(dataset_full)
dataset_full = dataset_full.merge(add_features, on='Company')
assert len(dataset_full) == length, 'MISMATCH'

### Create lag features and fillna smartly

In [47]:
dataset_full['l1_Last Price'] = dataset_full.groupby('Contract Name')['Last Price'].shift(1)
dataset_full['l2_Last Price'] = dataset_full.groupby('Contract Name')['Last Price'].shift(2)
dataset_full['l3_Last Price'] = dataset_full.groupby('Contract Name')['Last Price'].shift(3)
dataset_full['l4_Last Price'] = dataset_full.groupby('Contract Name')['Last Price'].shift(4)
dataset_full['l5_Last Price'] = dataset_full.groupby('Contract Name')['Last Price'].shift(5)
dataset_full['l6_Last Price'] = dataset_full.groupby('Contract Name')['Last Price'].shift(6)
dataset_full['l7_Last Price'] = dataset_full.groupby('Contract Name')['Last Price'].shift(7)
dataset_full['l8_Last Price'] = dataset_full.groupby('Contract Name')['Last Price'].shift(8)
dataset_full['l9_Last Price'] = dataset_full.groupby('Contract Name')['Last Price'].shift(9)
dataset_full['l10_Last Price'] = dataset_full.groupby('Contract Name')['Last Price'].shift(10)

dataset_full['l1_Stock_price'] = dataset_full.groupby('Contract Name')['Stock_price'].shift(1)
dataset_full['l2_Stock_price'] = dataset_full.groupby('Contract Name')['Stock_price'].shift(2)
dataset_full['l3_Stock_price'] = dataset_full.groupby('Contract Name')['Stock_price'].shift(3)
dataset_full['l4_Stock_price'] = dataset_full.groupby('Contract Name')['Stock_price'].shift(4)
dataset_full['l5_Stock_price'] = dataset_full.groupby('Contract Name')['Stock_price'].shift(5)
dataset_full['l6_Stock_price'] = dataset_full.groupby('Contract Name')['Stock_price'].shift(6)
dataset_full['l7_Stock_price'] = dataset_full.groupby('Contract Name')['Stock_price'].shift(7)
dataset_full['l8_Stock_price'] = dataset_full.groupby('Contract Name')['Stock_price'].shift(8)
dataset_full['l9_Stock_price'] = dataset_full.groupby('Contract Name')['Stock_price'].shift(9)
dataset_full['l10_Stock_price'] = dataset_full.groupby('Contract Name')['Stock_price'].shift(10)

dataset_full['l1_Stock_volume'] = dataset_full.groupby('Contract Name')['Stock_volume'].shift(1)
dataset_full['l2_Stock_volume'] = dataset_full.groupby('Contract Name')['Stock_volume'].shift(2)
dataset_full['l3_Stock_volume'] = dataset_full.groupby('Contract Name')['Stock_volume'].shift(3)
dataset_full['l4_Stock_volume'] = dataset_full.groupby('Contract Name')['Stock_volume'].shift(4)
dataset_full['l5_Stock_volume'] = dataset_full.groupby('Contract Name')['Stock_volume'].shift(5)
dataset_full['l6_Stock_volume'] = dataset_full.groupby('Contract Name')['Stock_volume'].shift(6)
dataset_full['l7_Stock_volume'] = dataset_full.groupby('Contract Name')['Stock_volume'].shift(7)
dataset_full['l8_Stock_volume'] = dataset_full.groupby('Contract Name')['Stock_volume'].shift(8)
dataset_full['l9_Stock_volume'] = dataset_full.groupby('Contract Name')['Stock_volume'].shift(9)
dataset_full['l10_Stock_volume'] = dataset_full.groupby('Contract Name')['Stock_volume'].shift(10)

In [48]:
dataset_full.isnull().sum()

Contract Name                   0
Strike                          0
Last Price                      0
Bid                             0
Ask                             0
Change                          0
% Change                        0
Volume                          0
Open Interest                   0
Company                         0
Date_get_data                   0
Date_expiration                 0
Time_to_maturity                0
r                               0
Stock_price                     0
Stock_volume                    0
Black_Scholes                   0
Option_price_volatility         0
Capitalization                  0
Age                             0
Weight                          0
l1_Last Price               37223
l2_Last Price               74446
l3_Last Price              111669
l4_Last Price              148892
l5_Last Price              186115
l6_Last Price              223338
l7_Last Price              260561
l8_Last Price              297784
l9_Last Price 

In [49]:
37223, 37223*2, 37223*3

(37223, 74446, 111669)

In [50]:
dataset_full['l1_Last Price'] = dataset_full['l1_Last Price'].fillna(dataset_full.groupby('Contract Name')['Last Price'].transform('median'))
dataset_full['l2_Last Price'] = dataset_full['l2_Last Price'].fillna(dataset_full.groupby('Contract Name')['Last Price'].transform('median'))
dataset_full['l3_Last Price'] = dataset_full['l3_Last Price'].fillna(dataset_full.groupby('Contract Name')['Last Price'].transform('median'))
dataset_full['l4_Last Price'] = dataset_full['l4_Last Price'].fillna(dataset_full.groupby('Contract Name')['Last Price'].transform('median'))
dataset_full['l5_Last Price'] = dataset_full['l5_Last Price'].fillna(dataset_full.groupby('Contract Name')['Last Price'].transform('median'))
dataset_full['l6_Last Price'] = dataset_full['l6_Last Price'].fillna(dataset_full.groupby('Contract Name')['Last Price'].transform('median'))
dataset_full['l7_Last Price'] = dataset_full['l7_Last Price'].fillna(dataset_full.groupby('Contract Name')['Last Price'].transform('median'))
dataset_full['l8_Last Price'] = dataset_full['l8_Last Price'].fillna(dataset_full.groupby('Contract Name')['Last Price'].transform('median'))
dataset_full['l9_Last Price'] = dataset_full['l9_Last Price'].fillna(dataset_full.groupby('Contract Name')['Last Price'].transform('median'))
dataset_full['l10_Last Price'] = dataset_full['l10_Last Price'].fillna(dataset_full.groupby('Contract Name')['Last Price'].transform('median'))


dataset_full['l1_Stock_price'] = dataset_full['l1_Stock_price'].fillna(dataset_full.groupby('Contract Name')['Stock_price'].transform('median'))
dataset_full['l2_Stock_price'] = dataset_full['l2_Stock_price'].fillna(dataset_full.groupby('Contract Name')['Stock_price'].transform('median'))
dataset_full['l3_Stock_price'] = dataset_full['l3_Stock_price'].fillna(dataset_full.groupby('Contract Name')['Stock_price'].transform('median'))
dataset_full['l4_Stock_price'] = dataset_full['l4_Stock_price'].fillna(dataset_full.groupby('Contract Name')['Stock_price'].transform('median'))
dataset_full['l5_Stock_price'] = dataset_full['l5_Stock_price'].fillna(dataset_full.groupby('Contract Name')['Stock_price'].transform('median'))
dataset_full['l6_Stock_price'] = dataset_full['l6_Stock_price'].fillna(dataset_full.groupby('Contract Name')['Stock_price'].transform('median'))
dataset_full['l7_Stock_price'] = dataset_full['l7_Stock_price'].fillna(dataset_full.groupby('Contract Name')['Stock_price'].transform('median'))
dataset_full['l8_Stock_price'] = dataset_full['l8_Stock_price'].fillna(dataset_full.groupby('Contract Name')['Stock_price'].transform('median'))
dataset_full['l9_Stock_price'] = dataset_full['l9_Stock_price'].fillna(dataset_full.groupby('Contract Name')['Stock_price'].transform('median'))
dataset_full['l10_Stock_price'] = dataset_full['l10_Stock_price'].fillna(dataset_full.groupby('Contract Name')['Stock_price'].transform('median'))


dataset_full['l1_Stock_volume'] = dataset_full['l1_Stock_volume'].fillna(dataset_full.groupby('Contract Name')['Stock_volume'].transform('median'))
dataset_full['l2_Stock_volume'] = dataset_full['l2_Stock_volume'].fillna(dataset_full.groupby('Contract Name')['Stock_volume'].transform('median'))
dataset_full['l3_Stock_volume'] = dataset_full['l3_Stock_volume'].fillna(dataset_full.groupby('Contract Name')['Stock_volume'].transform('median'))
dataset_full['l4_Stock_volume'] = dataset_full['l4_Stock_volume'].fillna(dataset_full.groupby('Contract Name')['Stock_volume'].transform('median'))
dataset_full['l5_Stock_volume'] = dataset_full['l5_Stock_volume'].fillna(dataset_full.groupby('Contract Name')['Stock_volume'].transform('median'))
dataset_full['l6_Stock_volume'] = dataset_full['l6_Stock_volume'].fillna(dataset_full.groupby('Contract Name')['Stock_volume'].transform('median'))
dataset_full['l7_Stock_volume'] = dataset_full['l7_Stock_volume'].fillna(dataset_full.groupby('Contract Name')['Stock_volume'].transform('median'))
dataset_full['l8_Stock_volume'] = dataset_full['l8_Stock_volume'].fillna(dataset_full.groupby('Contract Name')['Stock_volume'].transform('median'))
dataset_full['l9_Stock_volume'] = dataset_full['l9_Stock_volume'].fillna(dataset_full.groupby('Contract Name')['Stock_volume'].transform('median'))
dataset_full['l10_Stock_volume'] = dataset_full['l10_Stock_volume'].fillna(dataset_full.groupby('Contract Name')['Stock_volume'].transform('median'))

In [51]:
dataset_full.isnull().sum()

Contract Name              0
Strike                     0
Last Price                 0
Bid                        0
Ask                        0
Change                     0
% Change                   0
Volume                     0
Open Interest              0
Company                    0
Date_get_data              0
Date_expiration            0
Time_to_maturity           0
r                          0
Stock_price                0
Stock_volume               0
Black_Scholes              0
Option_price_volatility    0
Capitalization             0
Age                        0
Weight                     0
l1_Last Price              0
l2_Last Price              0
l3_Last Price              0
l4_Last Price              0
l5_Last Price              0
l6_Last Price              0
l7_Last Price              0
l8_Last Price              0
l9_Last Price              0
l10_Last Price             0
l1_Stock_price             0
l2_Stock_price             0
l3_Stock_price             0
l4_Stock_price

In [52]:
dataset_full.to_csv('6_More_features_06_05.csv', index=None)

# Complete